<a href="https://colab.research.google.com/github/dbremont/Notas/blob/main/Ingenerare/Databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- SQL Lite Source Code
  - Build & Debug
- Virtual Machines
- Parser
- B-Tree ...
- Para instalar SQLite hay que primero instalar tlc
- Para ma;na tratar de ./configure && make en folder correctamente y ejecutar la shell y crear una tabla simple.

## Primario


* [15-445/645 Introduction to Database Systems](https://15445.courses.cs.cmu.edu/fall2019/)

* [15-721 Advanced Database Systems](https://15721.courses.cs.cmu.edu/spring2020/)

* [6.830/6.814: Database Systems](http://dsg.csail.mit.edu/6.830/index.php) 

* [6.033 Computer System Engineering](http://web.mit.edu/6.033/www/index.shtml)

## Secundario

* [CS6530: Graduate-level Database System](https://www.cs.utah.edu/~lifeifei/cs6530/)

* [CS 245 Principles of Data-Intensive Systems](http://web.stanford.edu/class/cs245/)

* [CS 276 / LING 286: Information Retrieval and Web Search](http://web.stanford.edu/class/cs276/)

* [A Celebration of Mike Stonebraker](https://www.youtube.com/playlist?list=PLSE8ODhjZXjZO802SjzqBFFlkuKglgbZD)

* [How To Compile SQLite](https://sqlite.org/howtocompile.html)